# Bin Packing

$
\begin{equation}
\begin{aligned}
& \text{Given a set of :}\\ 
& & \text{bins } S_1, S_2 \dots \text{with the same size } V\\
& & n \text{ items with sizes } a_1, \dots, a_n \text{ to pack} \\
& \text{Objective :} \\
& & \text{Find an integer minimal number of bins } B\\ 
\end{aligned}
\end{equation}
$
$
\begin{equation}
\begin{aligned}
& \underset{x}{\text{min}}
& & B = \sum_{i=1}^{n} y_i \\
& \text{s.t.}
& & B \geq 1 \\ 
& & & \sum_{i=1}^{n}a_{j} x_{ij} \leq Vy_i, \forall i \in \{1, \dots, n\} \\
& & &  \sum_{i=1}^{n}x_{ij} = 1, \forall j \in \{1, \dots, n\} \\
& & &  x_{ij} \in \{0,1\}, \forall i \in \{1, \dots, n\},  \forall j \in \{1, \dots, n\} \\
& & &  y_{i} \in \{0,1\}, \forall j \in \{1, \dots, n\} \\
\end{aligned}
\end{equation}
$

$
\text{where } y_{i}=1 \text{ if bin } i \text{ is used and } x_{ij}=1 \text{ if item }j \text{ is put into bin } i
$